In [306]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import json
#from flask_pymongo import PyMongo
from pymongo import MongoClient

In [307]:

# Use 'read_html' to scrape the facts table into a dataframe
df = pd.read_html('https://www.vegasinsider.com/nfl/odds/las-vegas/', flavor='html5lib')[10]
#df
#df[[0,2]]
#df = df[df[2].notna()]
df.dropna(subset=[6], inplace=True)
df = df[[0,2]]
df


,0,2
2,11/29 1:00 PM 255 N.Y. Giants 256 Cincinnati,-6½ -1045u-10
3,11/29 1:00 PM 257 Cleveland 258 Jacksonville,-7½ -1048½u-10
4,11/29 1:00 PM 259 Carolina 260 Minnesota,50u-10-3 -10
5,11/29 1:00 PM 261 Tennessee 262 Indianapolis,51½u-10-3 EV
6,11/29 1:00 PM 263 L.A. Chargers 264 Buffalo,51½u-10-4 -10
7,11/29 1:00 PM 265 Miami 266 N.Y. Jets,-7½ -1045½u-10
8,11/29 4:05 PM 267 New Orleans 268 Denver,-17 -1036½u-10
9,11/29 4:05 PM 269 San Francisco 270 L.A. Rams,44½u-10-5½ -10
10,11/29 4:25 PM 271 Kansas City 272 Tampa Bay,-3½ EV56½u-10
11,11/29 8:20 PM 273 Chicago 274 Green Bay,44½u-10-7½ -10


In [308]:
for key, value in df.iteritems(): 
    print(key, value) 


0 2      11/29 1:00 PM  255 N.Y. Giants  256 Cincinnati
3      11/29 1:00 PM  257 Cleveland  258 Jacksonville
4          11/29 1:00 PM  259 Carolina  260 Minnesota
5      11/29 1:00 PM  261 Tennessee  262 Indianapolis
6       11/29 1:00 PM  263 L.A. Chargers  264 Buffalo
7             11/29 1:00 PM  265 Miami  266 N.Y. Jets
8          11/29 4:05 PM  267 New Orleans  268 Denver
9     11/29 4:05 PM  269 San Francisco  270 L.A. Rams
10      11/29 4:25 PM  271 Kansas City  272 Tampa Bay
11          11/29 8:20 PM  273 Chicago  274 Green Bay
12       11/30 8:15 PM  275 Seattle  276 Philadelphia
15           12/06 1:00 PM  453 Cincinnati  454 Miami
16       12/06 1:00 PM  455 Indianapolis  456 Houston
17     12/06 1:00 PM  457 Jacksonville  458 Minnesota
20        12/06 1:00 PM  463 New Orleans  464 Atlanta
21        12/06 1:00 PM  465 Cleveland  466 Tennessee
23          12/06 4:05 PM  469 L.A. Rams  470 Arizona
26         12/06 8:20 PM  475 Denver  476 Kansas City
27      12/07 8:15 PM  477

## Try BeautifulSoup

In [309]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/jrichards/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [310]:
url = 'https://www.vegasinsider.com/nfl/odds/las-vegas/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css(".frodds-data-tbl", wait_time=3)
#print(browser.html)
# does this exist?

True

# Function to get the spread odds

In [311]:
def get_spread_odds(table_rows):
    
    results = []
    
    for row in table_rows:
    
        # place to put the odds once parsed
        odds = []
    
        # Get all <td> tag contents, which are the columns
        columns = row.find_all("td")
    
        # Get the anchor tag text. Only interested in column index 2, the "VI Consensus"
        a = columns[2].find("a").contents
        #print(a)
    
        # Now cones the hard part - parse this 
        parts = a[2].split('\xa0')
        #print(parts)
    
        # If there is only one text element, then need to split on the "-"
        if len(parts) == 1:
            parts[0] = parts[0].replace("½", ".6")
            #print(parts)
            parts = parts[0].split("-")
            parts[1] = '-' + parts[1]
            odds.append(parts[0])
            odds.append(parts[1])
            #print("##" + str(parts))
        else:
            parts[0] = parts[0].replace("½", ".5")
            odds.append(parts[0])
            odds.append(parts[1])
            #print(parts)
        
        
        # Now get the next odds from the second line
        parts = a[4].split('\xa0')
        #print(len(parts))
        # If there is only one text element, then need to split on the "-"
        if len(parts) == 1:
            parts[0] = parts[0].replace("½", ".6")
            #print(parts)
            parts = parts[0].split("-")
            parts[1] = '-' + parts[1]
            odds.append(parts[0])
            odds.append(parts[1].replace("\n", "").replace("\t",""))
            #print("##" + str(parts))
        else:
            parts[0] = parts[0].replace("½", ".5")
            odds.append(parts[0])
            odds.append(parts[1].replace("\n", "").replace("\t",""))
            #print(parts)
            
        results.append(odds)
    
    return results

# Function to get the team names and ids

In [312]:
def get_game_info(table_rows):
    
    results = []
    
    for row in table_rows:
        
        teams_info = []
        
        # Get all <td> tag contents, which are the columns
        columns = row.find_all("td")
        
        info = columns[0].contents
        id1 = info[3].replace('\n', '').replace('\t','').replace('\xa0', '')
        team1 = info[4].text
        id2 = info[6].replace('\n', '').replace('\t','').replace('\xa0', '')
        team2 = info[7].text
        #print(str(id1), team1, id2, team2)
        
        teams_info.append(id1)
        teams_info.append(team1)
        teams_info.append(id2)
        teams_info.append(team2)
        
        results.append(teams_info)
        
    return results

# Open the page and get the table defines with class frodds-data-tbl

In [313]:
html = browser.html
news_soup = soup(html, 'html.parser')
the_table = news_soup.select_one(".frodds-data-tbl")
#print(my_elem)
table_rows = my_elem.find_all("tr")

# Get the spread odds

In [314]:
odds = get_spread_odds(table_rows)
print(odds)

[['-3.5', '-10', '53u', '-10'], ['-1', '-10', '50.6u', '-10'], ['-6.5', '-10', '45u', '-10'], ['-7', '-11', '47.6o', '-22'], ['50u', '-10', '-3', '-10'], ['51u', '-10', '-3', 'EV'], ['51.6u', '-10', '-4', '-10'], ['-7.5', '-10', '45u', '-10'], ['-16.5', '-10', '36.6u', '-10'], ['44.6u', '-10', '-5.5', '-10'], ['-3.5', '-10', '56u', '-10'], ['44.6u', '-10', '-8', '-10'], ['-6.5', '-10', '48.6u', '-10'], ['41.6u', '-10', '-10', '-10']]


# Get the team names

In [315]:
info = get_game_info(table_rows)
info

[['251', 'Las Vegas', '252', 'Atlanta'],
 ['253', 'Arizona', '254', 'New England'],
 ['255', 'N.Y. Giants', '256', 'Cincinnati'],
 ['257', 'Cleveland', '258', 'Jacksonville'],
 ['259', 'Carolina', '260', 'Minnesota'],
 ['261', 'Tennessee', '262', 'Indianapolis'],
 ['263', 'L.A. Chargers', '264', 'Buffalo'],
 ['265', 'Miami', '266', 'N.Y. Jets'],
 ['267', 'New Orleans', '268', 'Denver'],
 ['269', 'San Francisco', '270', 'L.A. Rams'],
 ['271', 'Kansas City', '272', 'Tampa Bay'],
 ['273', 'Chicago', '274', 'Green Bay'],
 ['275', 'Seattle', '276', 'Philadelphia'],
 ['279', 'Baltimore', '280', 'Pittsburgh']]

# Create a list of dictionaries of the game info and spread

In [320]:
spread_list = []
for index in range(len(info)):
    
    game = {
        '_id' : info[index][0] + "_" + info[index][2],
        'id1' : info[index][0],
        'team1' : info[index][1],
        'id2' : info[index][2],
        'team2' : info[index][3],
    }
    spread_list.append(game)
    
print(spread_list)
    

[{'_id': '251_252', 'id1': '251', 'team1': 'Las Vegas', 'id2': '252', 'team2': 'Atlanta'}, {'_id': '253_254', 'id1': '253', 'team1': 'Arizona', 'id2': '254', 'team2': 'New England'}, {'_id': '255_256', 'id1': '255', 'team1': 'N.Y. Giants', 'id2': '256', 'team2': 'Cincinnati'}, {'_id': '257_258', 'id1': '257', 'team1': 'Cleveland', 'id2': '258', 'team2': 'Jacksonville'}, {'_id': '259_260', 'id1': '259', 'team1': 'Carolina', 'id2': '260', 'team2': 'Minnesota'}, {'_id': '261_262', 'id1': '261', 'team1': 'Tennessee', 'id2': '262', 'team2': 'Indianapolis'}, {'_id': '263_264', 'id1': '263', 'team1': 'L.A. Chargers', 'id2': '264', 'team2': 'Buffalo'}, {'_id': '265_266', 'id1': '265', 'team1': 'Miami', 'id2': '266', 'team2': 'N.Y. Jets'}, {'_id': '267_268', 'id1': '267', 'team1': 'New Orleans', 'id2': '268', 'team2': 'Denver'}, {'_id': '269_270', 'id1': '269', 'team1': 'San Francisco', 'id2': '270', 'team2': 'L.A. Rams'}, {'_id': '271_272', 'id1': '271', 'team1': 'Kansas City', 'id2': '272', '

In [321]:
# Use flask_pymongo to set up mongo connection
client = MongoClient(port=27017)

In [326]:
db = client.spread
db.spread.delete_many({}) # Delete any that exists
for info in spread_list:
    result = db.spread.insert_one(info)
    print(result)